In [14]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from tqdm import notebook
import time

In [15]:
df = pd.read_csv("data_export4.csv")
df.head()

,ChestBottom_avg_speed,ChestBottom_avg_jerk,ChestBottom_avg_energy,ChestBottom_min_distance_from_hand,ChestBottom_angular_range_motion,Hip_avg_speed,Hip_avg_jerk,Hip_avg_energy,Hip_min_distance_from_hand,Hip_angular_range_motion,...,RightThigh_angular_range_motion,RightUpperArm_avg_speed,RightUpperArm_avg_jerk,RightUpperArm_avg_energy,RightUpperArm_min_distance_from_hand,RightUpperArm_angular_range_motion,pain,worry,confidence,activity
0,0.452092,0.005942,0.182715,0.385528,0.037213,0.048939,0.000729,0.002108,0.039243,0.038359,...,0.058330,0.535850,0.010549,0.294101,0.288035,0.036038,6.0,3.0,4.0,Loading washing machine
1,0.468861,0.004878,0.180866,0.385528,0.044008,0.047741,0.000581,0.001847,0.039243,0.045951,...,0.067577,0.479358,0.007990,0.243226,0.248654,0.043290,5.0,1.0,4.0,Loading washing machine
2,0.530785,0.002026,0.211555,0.387199,0.046139,0.048568,0.000307,0.001824,0.039345,0.048944,...,0.069050,0.397358,0.004514,0.144184,0.205351,0.045490,5.0,3.0,4.0,Loading washing machine
3,0.352602,0.004411,0.106639,0.302078,0.034126,0.052011,0.000377,0.002061,0.039319,0.034143,...,0.040463,0.339485,0.009092,0.107357,0.197284,0.033659,7.0,4.0,3.0,Unloading washing machine
4,0.494594,0.003669,0.192674,0.287037,0.046064,0.056265,0.000341,0.002345,0.039319,0.070022,...,0.071677,0.481523,0.006553,0.199359,0.240319,0.041978,6.0,4.0,3.0,Unloading washing machine


In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 341 entries, 0 to 340
Data columns (total 34 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   ChestBottom_avg_speed                 341 non-null    float64
 1   ChestBottom_avg_jerk                  341 non-null    float64
 2   ChestBottom_avg_energy                341 non-null    float64
 3   ChestBottom_min_distance_from_hand    341 non-null    float64
 4   ChestBottom_angular_range_motion      341 non-null    float64
 5   Hip_avg_speed                         341 non-null    float64
 6   Hip_avg_jerk                          341 non-null    float64
 7   Hip_avg_energy                        341 non-null    float64
 8   Hip_min_distance_from_hand            341 non-null    float64
 9   Hip_angular_range_motion              341 non-null    float64
 10  RightForeArm_avg_speed                341 non-null    float64
 11  RightForeArm_avg_je

In [17]:
# Define the bin edges for categorizing pain levels
bin_edges = [-1, 3, 7, float('inf')]
bin_labels = ['Low', 'Medium', 'High']

# Categorize 'pain' values
df['pain_category'] = pd.cut(df['pain'], bins=bin_edges, labels=bin_labels, right=False)

In [18]:
df['pain_category'].value_counts()

pain_category
Medium    211
High       88
Low        42
Name: count, dtype: int64

In [19]:
# Data Preparation
X = df.drop(['pain', 'pain_category'], axis=1)
y = df['pain_category']

# Convert the 'pain_category' column to categorical
y = y.astype('category')


In [20]:
# One-hot encode the 'activity' column
X = pd.get_dummies(X, columns=['activity'], prefix='activity')

In [21]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [24]:
# Standardize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [27]:
# Convert data to PyTorch tensors
X_train_tensor = torch.tensor(X_train_scaled, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32)


In [28]:
# Model Evaluation
model.eval()
with torch.no_grad():
    y_pred_logits = model(X_test_tensor)
    
y_pred_probs = torch.softmax(y_pred_logits, dim=1)
y_pred_classes = torch.argmax(y_pred_probs, dim=1)

y_test_labels = y_test.cat.codes  # Convert categorical labels to integer codes

y_test_numpy = y_test_labels.numpy()
y_pred_classes_numpy = y_pred_classes.numpy()

# Calculate F1 score
f1 = f1_score(y_test_numpy, y_pred_classes_numpy, average='weighted')
print("Weighted F1 Score:", f1)


AttributeError: 'Series' object has no attribute 'numpy'

In [15]:
# Define a custom dataset
class CustomDataset(Dataset):
    def __init__(self, features, labels):
        self.features = features
        self.labels = labels
        
    def __len__(self):
        return len(self.features)
    
    def __getitem__(self, idx):
        return self.features[idx], self.labels[idx]

In [16]:
train_dataset = CustomDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

In [17]:
# Model Definition
class MLP(nn.Module):
    def __init__(self, input_size):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_size, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 1)
        
    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [18]:
# Instantiate the model
input_size = X_train_scaled.shape[1]
model = MLP(input_size)

# Define loss and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [20]:
# Model Training
num_epochs = 100
for epoch in range(num_epochs):
    model.train()
    for batch_x, batch_y in train_loader:
        optimizer.zero_grad()
        outputs = model(batch_x)
        loss = criterion(outputs, batch_y.unsqueeze(1))
        loss.backward()
        optimizer.step()
    
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')


Epoch [1/100], Loss: 0.2034
Epoch [2/100], Loss: 0.3444
Epoch [3/100], Loss: 0.5116
Epoch [4/100], Loss: 0.2681
Epoch [5/100], Loss: 0.1383
Epoch [6/100], Loss: 0.1713
Epoch [7/100], Loss: 0.1960
Epoch [8/100], Loss: 0.1516
Epoch [9/100], Loss: 0.1719
Epoch [10/100], Loss: 0.0793
Epoch [11/100], Loss: 0.1963
Epoch [12/100], Loss: 0.2763
Epoch [13/100], Loss: 0.2440
Epoch [14/100], Loss: 0.1451
Epoch [15/100], Loss: 0.1855
Epoch [16/100], Loss: 0.3911
Epoch [17/100], Loss: 0.2583
Epoch [18/100], Loss: 0.1203
Epoch [19/100], Loss: 0.1756
Epoch [20/100], Loss: 0.5196
Epoch [21/100], Loss: 0.2710
Epoch [22/100], Loss: 0.1483
Epoch [23/100], Loss: 0.1286
Epoch [24/100], Loss: 0.2276
Epoch [25/100], Loss: 0.2176
Epoch [26/100], Loss: 0.0949
Epoch [27/100], Loss: 0.1277
Epoch [28/100], Loss: 0.0626
Epoch [29/100], Loss: 0.3674
Epoch [30/100], Loss: 0.1580
Epoch [31/100], Loss: 0.2059
Epoch [32/100], Loss: 0.1167
Epoch [33/100], Loss: 0.1371
Epoch [34/100], Loss: 0.0637
Epoch [35/100], Loss: 0

In [22]:
# Model Evaluation
model.eval()
with torch.no_grad():
    y_pred_tensor = model(X_test_tensor)
y_pred = y_pred_tensor.numpy()